In [338]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime as dt
pd.set_option('display.max_columns', 500)

In [340]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [341]:
data.shape

(188533, 13)

In [342]:
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [346]:
def extract_transmission_info(row):
    # Standardize transmission types
    row = row.upper()  # Make everything uppercase for consistency

    # Check for Automatic Transmission (A/T)
    if re.search(r'(A/T|AUTOMATIC|AUTO)', row):
        transmission_type = 'Automatic'
    # Check for Manual Transmission (M/T)
    elif re.search(r'(M/T|MANUAL)', row):
        transmission_type = 'Manual'
    # Check for CVT Transmission
    elif re.search(r'(CVT)', row):
        transmission_type = 'CVT'
    # Check for Single-Speed Transmission
    elif re.search(r'(1-SPEED|SINGLE-SPEED)', row):
        transmission_type = 'Single-Speed'
    else:
        transmission_type = 'Other'

    # Extract the number of speeds
    match = re.search(r'(\d+)-SPEED', row)
    if match:
        speeds = int(match.group(1))  # Convert to integer
    else:
        speeds = None  # If no speed is mentioned, set to None

    # Return as separate values
    return transmission_type, speeds

# Assuming `data['transmission']` contains the transmission data
data[['transmission_type', 'number_of_gears']] = data['transmission'].apply(
    lambda row: pd.Series(extract_transmission_info(row))
)
test[['transmission_type', 'number_of_gears']] = test['transmission'].apply(
    lambda row: pd.Series(extract_transmission_info(row))
)

In [347]:
data['clean_title'] = data.clean_title.fillna('No')
test['clean_title'] = test.clean_title.fillna('No')

In [348]:
len(data.model.unique())

1897

In [349]:
def extract_hp(row):
    match = re.search(r'(\d+\.?\d*)HP', row)
    return float(match.group(1)) if match else None

def extract_engine_size(row):
    match = re.search(r'(\d+\.?\d*)L', row)
    return float(match.group(1)) if match else None

def extract_cylinders(row):
    # First, check for explicit "Cylinder" mentions
    match = re.search(r'(\d+)\s*Cylinders?|V(\d+)\s*Cylinders?', row, re.IGNORECASE)
    if match:
        return int(match.group(1)) if match.group(1) else int(match.group(2))
    
    # If no "Cylinder" mentions, check for "V6" or "Inline4" etc.
    match = re.search(r'V(\d+)|Inline(\d+)', row, re.IGNORECASE)
    if match:
        return int(match.group(1)) if match.group(1) else int(match.group(2))
    
    return None  # Return None if no match is found

def extract_fuel_type(row):
    match = re.search(r'(Gasoline|Diesel|Flex\sFuel|Electric)', row, re.IGNORECASE)
    return match.group(1) if match else None

#def extract_engine_type(row):
#    match = re.search(r'(V\d+|Inline)', row, re.IGNORECASE)
#    return match.group(1) if match else None

def extract_fuel_capability(row):
    if 'Flex Fuel' in row:
        return 'Flex Fuel'
    elif 'Hybrid' in row:
        return 'Hybrid'
    elif 'Electric' in row:
        return 'Electric'
    else:
        return None

# Apply the functions to extract the information from the rows
data['HP'] = data['engine'].apply(extract_hp)
data['Engine_Size_L'] = data['engine'].apply(extract_engine_size)
data['Cylinders'] = data['engine'].apply(extract_cylinders)
data['Fuel_Type'] = data['engine'].apply(extract_fuel_type)
data['Fuel_Capability'] = data['engine'].apply(extract_fuel_capability)

test['HP'] = test['engine'].apply(extract_hp)
test['Engine_Size_L'] = test['engine'].apply(extract_engine_size)
test['Cylinders'] = test['engine'].apply(extract_cylinders)
test['Fuel_Type'] = test['engine'].apply(extract_fuel_type)
test['Fuel_Capability'] = test['engine'].apply(extract_fuel_capability)

In [350]:
data['logprice'] = np.log(data.price)

In [351]:
# Function to extract useful information
def extract_car_info(model):
    info = {}
    # Extract trim (assume the rest is the trim or variant)
    trim_pattern = re.search(r'(SE|LE|Limited|Sport|Premium|Base|Platinum|P\d+)', model, re.IGNORECASE)
    info['trim'] = trim_pattern.group(0) if trim_pattern else None
    
    # Extract drivetrain
    if 'xDrive' in model:
        info['drivetrain'] = 'AWD'
    elif '4MATIC' in model or '4WD' in model:
        info['drivetrain'] = '4WD'
    else:
        info['drivetrain'] = 'FWD'  # Fallback assumption
    
    return info

# Apply extraction
data_info = data['model'].apply(extract_car_info)
test_info = test['model'].apply(extract_car_info)


# Show extracted information
data[['trim', 'drivetrain']] = pd.DataFrame(data_info.tolist())
test[['trim', 'drivetrain']] = pd.DataFrame(test_info.tolist())

In [352]:
luxury_brands = ['Mercedes-Benz', 'Land',
                 'Rivian', 'Alfa', 'Porsche', 'McLaren',
                 'Jaguar', 'Rolls-Royce', 'Maserati',
                 'Bentley', 'Ferrari', 'Aston', 'Lamborghini', 'Lucid', 'Lotus', 'Bugatti', 'Maybach']

data['luxury'] = data['brand'].apply(lambda x: 1 if x in luxury_brands else 0)
test['luxury'] = test['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

In [353]:
year = 2024

data['age'] = 2024 - data['model_year']
data['mpy'] = data['milage'] / (data['age'] + 1)

test['age'] = 2024 - test['model_year']
test['mpy'] = test['milage'] / (test['age'] + 1)

In [354]:
data.mpy.max()

117500.0

In [392]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor

In [356]:
X = data.drop(['id', 'engine', 'price', 'logprice', 'transmission', 'model'], axis=1)

In [357]:
y = data['price']


num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1337)

In [426]:
num_trans = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean', add_indicator=True)),
    ('scale', StandardScaler())
])
cat_trans = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', OrdinalEncoder())
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_trans, num_cols),
    ('cat', cat_trans, cat_cols)
])

model3 = RandomForestRegressor(n_estimators=100, max_features=0.7)
#model = LinearRegression()
#model = Lasso(alpha=0.1)
#model = Ridge(alpha=0.1)
model2 = XGBRegressor(n_estimators=50, learning_rate=0.1, random_state=42)
model1 = LGBMRegressor(n_estimators=300, learning_rate=0.05, random_state=42)

pipeline1 = Pipeline(steps=[
    ('process', preprocessor),
    ('model', model1)
])
pipeline2 = Pipeline(steps=[
    ('process', preprocessor),
    ('model', model2)
])


In [428]:
base_models = [
    ('rf', pipeline1),
    ('xgb', pipeline2)
]

# Define the meta-model (Level-2 model)
meta_model = Ridge(alpha=0.1)

# Define the stacking regressor
stacked_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

# Fit the stacked model
stacked_model.fit(X, y)

# Make predictions
y_pred_stacked = stacked_model.predict(X)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 188533, number of used features: 26
[LightGBM] [Info] Start training from score 43878.016178
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021990 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1362
[LightGBM] [Info] Number of data points in the train set: 150826, number of used features: 26
[LightGBM] [Info] Start training from score 43829.197671
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011344 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info

In [430]:
preds = stacked_model.predict(X_train)
root_mean_squared_error(preds, y_train)

68003.90998146245

In [432]:
preds = stacked_model.predict(X_test)
root_mean_squared_error(preds, y_test)

73593.86914276036

In [434]:
test = test.set_index('id')
test = test.drop(['engine', 'transmission', 'model'], axis=1)

KeyError: "['engine', 'transmission', 'model'] not found in axis"

In [436]:
preds = stacked_model.predict(test)

In [437]:
test = test.reset_index()
ids = test['id']

In [440]:
submission_df = pd.DataFrame({'id' : ids, 'price' : preds})
submission_df.set_index('id')

,price
id,
188533,19942.961756
188534,83170.137747
188535,73794.772170
188536,30138.870150
188537,30781.961946
...,...
314218,29514.620224
314219,46218.482325
314220,25025.906490


In [442]:
submission_df.to_csv('submission.csv', index=False)